In [20]:
import geopandas as gpd
import pandas as pd
import sys
import numpy as np
import rasterio as rs
from rasterio.mask import mask
from rasterio.warp import calculate_default_transform, reproject, Resampling
#from rasterstats import zonal_stats

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [21]:
def reproject_to_meters(input_raster, output_raster, target_crs='EPSG:3857'):
    
    with rs.open(input_raster) as src:
        transform, width, height = calculate_default_transform(
            src.crs, target_crs, src.width, src.height, *src.bounds
        )
        kwargs = src.meta.copy()
        kwargs.update({
            'crs': target_crs,
            'transform': transform,
            'width': width,
            'height': height
        })

        with rs.open(output_raster, 'w', **kwargs) as dst:
            for i in range(1, src.count + 1):
                reproject(
                    source=rs.band(src, i),
                    destination=rs.band(dst, i),
                    src_transform=src.transform,
                    src_crs=src.crs,
                    dst_transform=transform,
                    dst_crs=target_crs,
                    resampling=Resampling.nearest
                )

In [22]:
prj_districts = gpd.read_file('../../data/epa_districts/Project_Districts.shp')
input_raster = 'final_2024-10-29.tif'
repj_raster = 'reprojected_2024-10-29.tif'

# reproject inputs from degrees to meters for area assessment calcs
dst_crs = 'EPSG:3857'
reproject_to_meters(input_raster, repj_raster, dst_crs)
prj_districts = prj_districts.to_crs(dst_crs)

In [26]:
zonal_stats = []

with rs.open(repj_raster) as src:
    for _, district in prj_districts.iterrows():
        out_image, out_transform = mask(src, [district.geometry], crop=True)
        district_mask = out_image[0] 
        unique, counts = np.unique(district_mask, return_counts=True)
        land_use_stats = dict(zip(unique, counts))
        land_use_stats = {k: round((v / 10000),2) for k, v in land_use_stats.items()}
        land_use_stats['district'] = district.ADM2_EN
        zonal_stats.append(land_use_stats)

df = pd.DataFrame(zonal_stats)
df = df.rename(columns= {
                        0: "no_tree",
                        1: "monoculture",
                        2: "agroforestry",
                        3: "natural",
                        255: "no_data"
                    })

In [27]:
df

,no_tree,monoculture,agroforestry,natural,no_data,district
0,57.98,10.44,523.43,161.83,563.76,Adansi South
1,44.88,6.38,949.29,143.95,1321.85,Asante Akim South
2,32.70,7.10,649.82,28.85,505.52,Assin North
3,111.53,49.89,1551.57,155.77,1638.30,Atwima Mponua
4,895.44,0.05,161.81,23.17,876.60,Bawku West
5,34.43,7.24,405.99,116.19,957.82,Bosome Freho
6,970.92,0.01,251.15,36.12,1902.87,Builsa South
7,1111.67,0.00,173.96,187.94,1388.50,Daffiama Bussie Issa
8,13.38,5.35,120.70,30.54,394.62,Juaben Municipal
9,665.57,0.12,196.23,24.61,1423.15,Kasena Nankana West


In [ ]:
# df['ag_area'] = df.agroforestry + df.monoculture
# df['nat_area'] = df.natural
# df = df[['district', 'ag_area', 'nat_area']]
# df

In [4]:
## Double check w/ Rasterstats Approach
lu_cats = {
    0: "no_tree",
    1: "monoculture",
    2: "agroforestry",
    3: "natural",
    255: "no_data"
}
stats = zonal_stats('../../data/epa_districts/Project_Districts.shp',
           'final_2024-10-29.tif',
            categorical=True, 
            category_map=lu_cats)